In [1]:
import os
dirname = "../data/parse/markdown_v2"

os.listdir(dirname)

['2023-04-17 Quy định tổ chức thi tiếng Anh nội bộ.md',
 '2023-06-01 Quyết đinh ban hành quy chế đào tạo 2023.md',
 '2892- quy định đánh giá các TCCSĐ trực thuộc ĐUK 2023 (bản PH).md',
 '3. Quy che QLSD tai san ĐHBKHN_24_5-2023 final.md',
 '5. Huong dan quy trinh phat trien dang vien.md',
 '5776.2023 QĐ miễn giảm học phí 2023.md',
 '5777.2023 Quy định QLSV nước ngoài 2023.md',
 '5778.2023 QĐ Học bổng KKHT 2023.md',
 '5847.2023 QĐ Học bổng Trần Đại Nghĩa 2023.md',
 '5848.2023 QĐ thi Olympic và ĐMST 2023.md',
 '5880.2023 QĐ đánh giá điểm rèn luyện sinh viên 2023.md',
 '6008.2023 QĐ Quản lý câu lạc bộ sinh viên 2023.md',
 'NQ.03.2024_0001_Quy chế Tổ chức và hoạt động_đã ký.md',
 'NQ.04.2024_ Quy chế Quản lý tài chính_đã ký.md',
 'NQ.05.2024_Quy chế Dân chủ_đã ký.md',
 'QC tuyển sinh ĐH 2023_Final_19.5.2023.md',
 'QD 2891 ĐUK, ngay 23.11.2023 ve danh gia can bo LDQL (PH).md',
 'qui-dinh-ve-xttn-nam-2024_final_2.md',
 'Quy dinh QL HĐHTĐN_HUST.md',
 'Quy định xét cấp HB tài trợ 2024 LasVer.m

In [87]:
path = os.path.join(dirname, os.listdir(dirname)[13])
with open(path, "r") as f:
    content = f.read()


In [88]:
import re
from typing import List, Dict, Any, Optional

def extract_articles(text: str) -> List[Dict[str, Any]]:

    # Pattern to match articles
    article_pattern = re.compile(
        r'#\s*(?:ĐIỀU|Điều)\s+(\d+)(?:\.|\s*\.)?\s*([^\n]+)?\n(.*?)(?=(?:#\s*(?:ĐIỀU|Điều|CHƯƠNG|Chương|MỤC|Mục)|\Z))',
        re.DOTALL | re.IGNORECASE
    )
    
    # Pattern for numbered paragraphs
    paragraph_pattern = re.compile(r'^\s*(\d+)[\.:]\s*(.+?)(?=\n\s*\d+[\.:]\s*|\Z)', re.DOTALL | re.MULTILINE)
    
    articles = []
    for match in article_pattern.finditer(text):
        article_num = match.group(1)
        article_title = match.group(2).strip() if match.group(2) else ""
        article_content = match.group(3).strip()
        
        # Extract paragraphs
        paragraphs = {}
        for para_match in paragraph_pattern.finditer(article_content):
            para_num = para_match.group(1)
            para_content = para_match.group(2).strip()
            paragraphs[para_num] = para_content
        
        articles.append({
            "number": article_num,
            "title": article_title,
            "content": article_content,
            "paragraphs": paragraphs
        })
    
    return articles

def extract_sections(text: str) -> List[Dict[str, Any]]:

    # Pattern to match sections
    section_pattern = re.compile(
        r'#\s*(?:MỤC|Mục)\s+([IVXLCDM0-9]+)(?:[\.:]?\s*([^\n]+))?',
        re.IGNORECASE
    )
    
    sections = []
    section_matches = list(section_pattern.finditer(text))
    
    for i, match in enumerate(section_matches):
        section_num = match.group(1)
        section_title = match.group(2).strip() if match.group(2) else ""
        
        # Determine section content range
        start_pos = match.end()
        end_pos = len(text)
        if i < len(section_matches) - 1:
            end_pos = section_matches[i + 1].start()
        
        section_content = text[start_pos:end_pos]
        
        # Extract articles in this section
        section_articles = extract_articles(section_content)
        
        sections.append({
            "number": section_num,
            "title": section_title,
            "articles": section_articles
        })
    
    return sections

def extract_chapters(text: str) -> List[Dict[str, Any]]:

    # Pattern to match chapters
    chapter_pattern = re.compile(
        r'#\s*(?:(?:CHƯƠNG|Chương)\s+([IVXLCDM]+)|([IVXLCDM]+)(?:-|\s+-\s*))(?:\s*(.+?))?(?=\n|$)',
        
        re.IGNORECASE
    )
    
    chapters = []
    chapter_matches = list(chapter_pattern.finditer(text))
    
    for i, match in enumerate(chapter_matches):
        chapter_num = match.group(1) if match.group(1) else match.group(2)
        chapter_title = match.group(3).strip() if match.group(3) else ""
        
        # Check for title on next line if not in heading
        if not chapter_title:
            title_pattern = re.compile(r'#\s*([^\n#]+)')
            next_line = text[match.end():].lstrip()
            title_match = title_pattern.match(next_line)
            if title_match:
                chapter_title = title_match.group(1).strip()
        
        # Determine chapter content range
        start_pos = match.end()
        end_pos = len(text)
        if i < len(chapter_matches) - 1:
            end_pos = chapter_matches[i + 1].start()
        
        chapter_content = text[start_pos:end_pos]
        
        # Extract sections in this chapter
        chapter_sections = extract_sections(chapter_content)
        
        # If no sections, extract articles directly
        chapter_articles = []
        if not chapter_sections:
            chapter_articles = extract_articles(chapter_content)
        
        chapters.append({
            "number": chapter_num,
            "title": chapter_title,
            "sections": chapter_sections,
            "articles": chapter_articles
        })
    
    return chapters

def parse_document(text: str) -> Dict[str, Any]:

    # Try to extract chapters
    chapters = extract_chapters(text)
    
    # If no chapters, try sections
    sections = []
    articles = []
    if not chapters:
        sections = extract_sections(text)
        
        # If no sections either, extract articles directly
        if not sections:
            articles = extract_articles(text)
    
    return {
        "chapters": chapters,
        "sections": sections,
        "articles": articles
    }

def get_chunks_with_metadata(text: str) -> List[Dict[str, Any]]:

    parsed_doc = parse_document(text)
    chunks = []
    
    # Process articles directly in document
    for article in parsed_doc["articles"]:
        chunks.append({
            "text": article["content"],
            "metadata": {
                "article_number": article["number"],
                "article_title": article["title"],
                "chapter_number": None,
                "chapter_title": None,
                "section_number": None,
                "section_title": None
            }
        })
    
    
    # Process articles in chapters
    for chapter in parsed_doc["chapters"]:
        # Articles directly in chapter
        for article in chapter["articles"]:
            chunks.append({
                "text": article["content"],
                "metadata": {
                    "article_number": article["number"],
                    "article_title": article["title"],
                    "chapter_number": chapter["number"],
                    "chapter_title": chapter["title"],
                    "section_number": None,
                    "section_title": None
                }
            })
        
        # Articles in sections within chapters
        for section in chapter["sections"]:
            for article in section["articles"]:
                chunks.append({
                    "text": article["content"],
                    "metadata": {
                        "article_number": article["number"],
                        "article_title": article["title"],
                        "chapter_number": chapter["number"],
                        "chapter_title": chapter["title"],
                        "section_number": section["number"],
                        "section_title": section["title"]
                    }
                })
    
    return chunks


chunks = get_chunks_with_metadata(content)

# Print result
import json
print(json.dumps(chunks, ensure_ascii=False, indent=2))

[
  {
    "text": "",
    "metadata": {
      "article_number": "13",
      "article_title": "Giá dịch vụ đào tạo",
      "chapter_number": "III",
      "chapter_title": "QUẢN LÝ VÀ PHÂN BỎ THU",
      "section_number": null,
      "section_title": null
    }
  },
  {
    "text": "",
    "metadata": {
      "article_number": "14",
      "article_title": "Hợptác dào tạo vànghiêncứu",
      "chapter_number": "III",
      "chapter_title": "QUẢN LÝ VÀ PHÂN BỎ THU",
      "section_number": null,
      "section_title": null
    }
  },
  {
    "text": "",
    "metadata": {
      "article_number": "15",
      "article_title": "Kinh doanh và dịch vụ.",
      "chapter_number": "III",
      "chapter_title": "QUẢN LÝ VÀ PHÂN BỎ THU",
      "section_number": null,
      "section_title": null
    }
  },
  {
    "text": "",
    "metadata": {
      "article_number": "16",
      "article_title": "Tài trợ và tặng quà",
      "chapter_number": "III",
      "chapter_title": "QUẢN LÝ VÀ PHÂN BỎ THU",
     

In [ ]:

for chunk in chunks:
    metadata = chunk['metadata']
    print(f"Chương {metadata['chapter_number']}: {metadata['chapter_title']}, Mục {metadata['section_number']}: {metadata['section_title']}, Điều {metadata['article_number']}: {metadata['article_title']}")
    print("--------")
    print(chunk['text'])
    print("-"*40)

In [77]:
from langchain_core.documents import Document

md_dir = "../data/parse/markdown_v2/"

def split_md(dir, **kwargs):
    doc_list = []
    for file in os.listdir(dir):
        full_path = os.path.join(md_dir, file)
        with open(full_path, "r") as f:
            content = f.read()

        chunks = get_chunks_with_metadata(content)

        for chunk in chunks:
            chunk_metadata = chunk["metadata"]
            chunk_metadata["source"] = file
            doc_list.append(Document(page_content=chunk["text"], metadata=chunk_metadata, **kwargs))

    return doc_list

doc_list = split_md(md_dir)

In [78]:
print(len(doc_list))

443


In [86]:
for i, doc in enumerate(doc_list):
    if not doc.page_content:
        print(doc.metadata)

{'article_number': '13', 'article_title': 'Giá dịch vụ đào tạo', 'chapter_number': 'III', 'chapter_title': 'QUẢN LÝ VÀ PHÂN BỎ THU', 'section_number': None, 'section_title': None, 'source': 'NQ.04.2024_ Quy chế Quản lý tài chính_đã ký.md'}
{'article_number': '14', 'article_title': 'Hợptác dào tạo vànghiêncứu', 'chapter_number': 'III', 'chapter_title': 'QUẢN LÝ VÀ PHÂN BỎ THU', 'section_number': None, 'section_title': None, 'source': 'NQ.04.2024_ Quy chế Quản lý tài chính_đã ký.md'}
{'article_number': '15', 'article_title': 'Kinh doanh và dịch vụ.', 'chapter_number': 'III', 'chapter_title': 'QUẢN LÝ VÀ PHÂN BỎ THU', 'section_number': None, 'section_title': None, 'source': 'NQ.04.2024_ Quy chế Quản lý tài chính_đã ký.md'}
{'article_number': '16', 'article_title': 'Tài trợ và tặng quà', 'chapter_number': 'III', 'chapter_title': 'QUẢN LÝ VÀ PHÂN BỎ THU', 'section_number': None, 'section_title': None, 'source': 'NQ.04.2024_ Quy chế Quản lý tài chính_đã ký.md'}
{'article_number': '17', 'arti

In [83]:
import re
from typing import List, Dict, Any, Tuple

def get_content_between_headings(text: str, heading_pattern: str) -> Tuple[List[Dict[str, Any]], str]:
    """
    Extract content between headings that match the given pattern,
    and return the remaining content that doesn't belong to any matching heading.
    
    Args:
        text: Text content to process
        heading_pattern: Regex pattern for the headings to find
        
    Returns:
        Tuple of (list of heading matches with content, remaining content)
    """
    pattern = re.compile(heading_pattern, re.DOTALL)
    matches = list(pattern.finditer(text))
    results = []
    remaining_content = text
    
    if matches:
        # Process each heading match
        for i, match in enumerate(matches):
            heading_start = match.start()
            heading_end = match.end()
            
            # Determine content end
            content_end = len(text)
            if i < len(matches) - 1:
                content_end = matches[i+1].start()
            
            # Extract the heading and its content
            full_content = text[heading_start:content_end]
            heading_text = match.group(0)
            
            # Store the result
            results.append({
                "match": match,
                "full_text": full_content,
                "heading": heading_text
            })
        
        # Determine remaining content (text before the first heading)
        if matches[0].start() > 0:
            remaining_content = text[:matches[0].start()]
        else:
            remaining_content = ""
    
    return results, remaining_content

def extract_sections(text: str) -> List[Dict[str, Any]]:
    """
    Extract sections (in format like "# 1.1 Title") and their content.
    
    Args:
        text: Text content to process
        
    Returns:
        List of section dictionaries with number, title, and content
    """
    # Get all section headings and content
    section_pattern = r'#\s*(\d+\.\d+)\s+([^\n]+)'
    section_matches, remaining_text = get_content_between_headings(text, section_pattern)
    
    sections = []
    for section_data in section_matches:
        match = section_data["match"]
        section_num = match.group(1)
        section_title = match.group(2).strip() if match.group(2) else ""
        full_text = section_data["full_text"]
        heading_text = section_data["heading"]
        
        # Extract the section content (excluding the heading)
        section_content = full_text[len(heading_text):].strip()
        
        sections.append({
            "number": section_num,
            "title": section_title,
            "content": section_content
        })
    
    return sections

def extract_chapters(text: str) -> List[Dict[str, Any]]:
    """
    Extract chapters (in format like "# I. Title") and their contained sections.
    
    Args:
        text: Text content to process
        
    Returns:
        List of chapter dictionaries with number, title, content, and sections
    """
    # Get all chapter headings and content
    # This pattern matches both "# I. Title" and "# I- Title" formats
    chapter_pattern = r'#\s*([IVXLCDM]+)(?:\.|-)\s+([^\n]+)'
    chapter_matches, remaining_text = get_content_between_headings(text, chapter_pattern)
    
    chapters = []
    for chapter_data in chapter_matches:
        match = chapter_data["match"]
        chapter_num = match.group(1)
        chapter_title = match.group(2).strip() if match.group(2) else ""
        full_text = chapter_data["full_text"]
        heading_text = chapter_data["heading"]
        
        # Extract the chapter content (excluding the heading)
        chapter_content = full_text[len(heading_text):].strip()
        
        # Extract sections in this chapter
        chapter_sections = extract_sections(full_text)
        
        # Calculate content directly belonging to the chapter (before any sections)
        direct_content = ""
        if chapter_sections:
            # Find where the first section starts
            first_section_pattern = r'#\s*\d+\.\d+'
            first_section_match = re.search(first_section_pattern, chapter_content)
            if first_section_match:
                direct_content = chapter_content[:first_section_match.start()].strip()
            else:
                direct_content = chapter_content
        else:
            direct_content = chapter_content
        
        chapters.append({
            "number": chapter_num,
            "title": chapter_title,
            "content": direct_content,
            "sections": chapter_sections
        })
    
    return chapters

def parse_document(text: str) -> Dict[str, Any]:
    """
    Parse a document with chapters (I.) and sections (1.1).
    
    Args:
        text: Document text to process
        
    Returns:
        Dictionary with document structure
    """
    # Extract all structural elements
    chapters = extract_chapters(text)
    
    return {
        "chapters": chapters
    }

def get_chunks_with_metadata(text: str) -> List[Dict[str, Any]]:
    """
    Process a document and return chunks with chapter and section metadata.
    
    Args:
        text: Document text to process
        
    Returns:
        List of chunks, each with text content and hierarchical metadata
    """
    parsed_doc = parse_document(text)
    chunks = []
    
    # Process chapters
    for chapter in parsed_doc["chapters"]:
        chapter_num = chapter["number"]
        chapter_title = chapter["title"]
        
        # Create a chunk for direct chapter content (if not empty)
        if chapter["content"]:
            chunks.append({
                "text": chapter["content"],
                "metadata": {
                    "chapter_number": chapter_num,
                    "chapter_title": chapter_title,
                    "section_number": None,
                    "section_title": None
                }
            })
        
        # Process sections in chapter
        for section in chapter["sections"]:
            section_num = section["number"]
            section_title = section["title"]
            
            chunks.append({
                "text": section["content"],
                "metadata": {
                    "chapter_number": chapter_num,
                    "chapter_title": chapter_title,
                    "section_number": section_num,
                    "section_title": section_title
                }
            })
    
    return chunks

chunks = get_chunks_with_metadata(content)

In [85]:
for chunk in chunks:
    metadata = chunk['metadata']
    print(f"Chương {metadata['chapter_number']}: {metadata['chapter_title']}, Mục {metadata['section_number']}: {metadata['section_title']}")
    print("--------")
    print(chunk['text'])
    print("-"*40)

Chương I: THỦ TỤC KẾT NẠP ĐẢNG VIÊN (KỂ CẢ KẾT NẠP LẠI)., Mục None: None
--------
Người vào Đảng phải:

- Có đơn tự nguyện xin vào Đảng.
- Báo cáo trung thực lý lịch với chi bộ sẽ xem xét thủ tục kết nạp Đảng cho mình.
- Được hai đảng viên chính thức giới thiệu hoặc được một đảng viên chính thức và Ban chấp hành (BCH) Đoàn cơ sở giới thiệu, người xin vào Đảng trong độ tuổi thanh niên phải là đoàn viên. Nếu người đã hết tuổi Đoàn mà còn sinh hoạt Đoàn hoặc giữ chức vụ trong tổ chức Đoàn thì phải có ý kiến nhận xét của tổ chức Đoàn. Người vào Đảng là cán bộ khi đã hết tuổi Đoàn phải là Công đoàn viên.
----------------------------------------
Chương I: THỦ TỤC KẾT NẠP ĐẢNG VIÊN (KỂ CẢ KẾT NẠP LẠI)., Mục 1.1: Quy trình công tác phát triển Đảng.
--------
# 1.1.1 Giới thiệu quần chúng ưu tú đi dự lớp bồi dưỡng nhận thức về Đảng (Đối tượng Đảng) theo chương trình quy định của Trung ương (người vào Đảng phải được học lớp bồi dưỡng nhận thức về Đảng và được cấp giấy chứng nhận).

Các bước tiến 